<a href="https://colab.research.google.com/github/svandergoote/LGBIO2060-2021/blob/TP4/LGBIO2060_TP4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LGBIO2060 Exercice session 4

#Hidden markov model

__Authors:__ Simon Vandergooten, Clémence Vandamme

__Content inspired from__: Neuromatch Academy github.com/NeuromatchAcademy


##Introduction and context
In this tutorial we will make the previous models a little more complex. Until now, we have used binary or continuous variables whose state was fixed in time. It is easy to see that this greatly limits our ability to represent real systems. One way to get closer to reality would be to allow our hidden states to change their values. 

Let's take the example of the fish. Previously they were either on the right or on the left. But now, we will add to the model the fact that fishes can switch side. 

It is exactly what **Hidden markov models** permits. The Markov property specifies that you can fully encapsulate the important properties of a system based on its current state at the current time, any previous history does not matter. It is memoryless. Back to the fishes, it means that the school of fish is only at one position at a time and the probability of them being on the left or on the right at time *t* depends only on the state at time *t-1* and the probabilities of transition from one state to another. It can be represented with a matrix called the **transition matrix**.

<img alt='Solution hint' align='left' width=650 height=300 src=https://raw.githubusercontent.com/svandergoote/LGBIO2060-2021/master/Solutions/HMM.png>


We can use linear algebra to express the probabilities of the current state.

$$P_i = [P(state_i = right), P(state_i = left) ] $$

To compute the vector of probabilities of the state at the time i+1, we can use linear algebra and multiply our vector of the probabilities of the current state with the transition matrix.

$$P_{i+1} = P_{i} T$$
where $T$ is our transition matrix.

This is the same formula for every step, which allows us to get the probabilities for a time more than 1 step in advance easily. If we started at $i=0$ and wanted to look at the probabilities at step $i=2$, we could do:

\begin{align*}
P_{1} &= P_{0}T\\
P_{2} &= P_{1}T = P_{0}TT = P_{0}T^2\\
\end{align*}

So, every time we take a further step we can just multiply with the transition matrix again. So, the probability vector of states at j timepoints after the current state at timepoint i is equal to the probability vector at timepoint i times the transition matrix raised to the jth power.
$$P_{i + j} = P_{i}T^j $$

By the end of this tutorial, you should be able to:
- Describe how the hidden states in a Hidden Markov model evolve over time, both in words, mathematically, and in code
- Estimate hidden states from data using forward inference in a Hidden Markov model
- Describe how measurement noise and state transition probabilities affect uncertainty in predictions in the future and the ability to estimate hidden states.

# Section 1: Binary HMM with Gaussian measurements
We will represent the hidden state *s* of the fish with the values +1 and -1 for the right and left position respectively. 

The probability of switching to state $s_t=j$ from the previous state $s_{t-1}=i$ is the conditional probability distribution $p(s_t = j| s_{t-1} = i)$.

In our case, we can summarize those transition probabilities into the **Transition matrix T**.

\begin{align*}
T = \begin{bmatrix}p(s_t = +1 | s_{t-1} = +1) & p(s_t = -1 | s_{t-1} = +1)\\p(s_t = +1 | s_{t-1} = -1)& p(s_t = -1 | s_{t-1} = -1)\end{bmatrix}
\end{align*}

###Measurements
In a Hidden Markov model, we cannot directly observe the latent states $s_t$. Instead we get noisy measurements $m_t \sim p(m|s_t)$

##Coding exercice 1.1 : Hidden states
You will first implement the function `generate_state`. This function will create the vector of states *S* based on the model parameters.

In [ ]:
def generate_state(switch_proba, start_proba, n_states):
  '''
  Create an HMM binary state variable.
  Args:
    switch_proba (array): the probabilities to switch from states. [rightToLeft, LeftToRight]
    start_proba (array): the initial probabilities of being on each side. [p_right, p_left]
    n_states (int): the number of time steps

  Returns:
    S (array): the vector of state for each time step.
  '''
  ##########################
  ##### Your code here #####
  ##########################
  #Initialize S
  S = ...

  #Step 1: Initial state (Hint: np.random.choice)
  S[0] = ...

  #Step 2: Transition matrix
  T = ...
  
  #Step 3: Iterate on each time step to find the new state s[t] based on S[t-1]



  return S

#Set random seed
np.random.seed(54)

#Set parameters of HMM
switch_proba = np.array([0.4, 0.7])
start_proba = np.array([1, 0]) #The initial state is right (+1)
n_states = 50

#Generate the hidden states vector
S = generate_state(switch_proba, start_proba, n_states)

##Coding exercice 1.2 : Noisy measurements
Now that you have created the hidden states, you will create the noisy Gaussian measurements vector *M* from it.

Recall that in reality we don't have access to the hidden states but only to noisy measurements that give us information about those hidden states we want to infer.

You will implement the function `sample` that generates samples $m_t$ based on the hidden states $s_t$. 
- if hidden state $s_t = +1 $ : $m_t \sim N(+1,\sigma)$
- if hidden state $s_t = -1 $ : $m_t \sim N(-1,\sigma)$



In [ ]:
def sample(means, var, S):
  '''
  Create a Gaussian measurement from HMM states

    Args: 
      means (array): Mean mesurement for each state [right, left].
      var (float): Variance of measurement models. Same for each state.
      S (array): The series of hidden states.
    
    Returns:
      M (array): The series of measurements.
  '''
  ##########################
  ##### Your code here #####
  ##########################

  #Calculate measurements conditioned on the latent states (Hint: np.random.normal)
  

  return M